# Fine Food Review

Ludovic Herbelin, 2020

**Goals : Predict review score from the text of the review.**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import numpy as np
from tqdm.notebook import tqdm
import sklearn

## Loading data

In [2]:
DATASET_PATH = 'Reviews.csv'
N_ROWS = 60000

dataset = pd.read_csv(DATASET_PATH, nrows=N_ROWS)

print(f"{len(dataset)} rows in data")

60000 rows in data


In [3]:
df = dataset.copy()
df = df.drop(columns=['Id','ProductId', 'UserId', 'ProfileName', 'Time', 'Text'])
feature_column = 'Summary'

df.dropna(inplace=True)
df.head()

,HelpfulnessNumerator,HelpfulnessDenominator,Score,Summary
0,1,1,5,Good Quality Dog Food
1,0,0,1,Not as Advertised
2,1,1,4,"""Delight"" says it all"
3,3,3,2,Cough Medicine
4,0,0,5,Great taffy


### Remove the neutral reviews

In [4]:
# depending on our goals we might want to remove neutral reviews (IE score of 3)
REMOVE_NEUTRAL = False

if REMOVE_NEUTRAL:
    df = df[df.Score != 3]
    print(f"{len(df)} rows left")

### Dataset preprocessing

In [5]:
X = df.loc[:, df.columns != 'Score']
Y = df['Score']

In [6]:
from sklearn.model_selection import train_test_split
TRAIN_RATIO = 0.8

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, train_size=TRAIN_RATIO, random_state=42)

print(f"Train size : {len(Y_train)}, test set size : {len(Y_test)}")

Train size : 47998, test set size : 12000


### Text preprocessing

Helper class for preprocessing the text : remove stopwords, punctuation etc.

In [7]:
from nltk.stem import SnowballStemmer 
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords


class TextProcessor:
    def __init__(self, language='english', remove_stopwords = True, perform_stemming = True):
        """ init functions, load the stopwords and punkt for tokenizer
        Keywords args :
        language -- 'english' or other
        remove_stopwords -- If true, will perform stopwords removal
        perform_stemming -- If true, will stem the words
        """
        nltk.download('stopwords')
        nltk.download('punkt')
        self.perform_stemming = perform_stemming
        if(self.perform_stemming):
            self.stemmer = SnowballStemmer(language)
        self.remove_stopwords = remove_stopwords
        if(self.remove_stopwords):
            self.stopwords = set(stopwords.words(language))  

    def __stem_word__(self, word):
        return self.stemmer.stem(word)

    def __tokenize_text__(self, text):
        return word_tokenize(text)


    def preprocess_text(self, text):
        """Set the words in the text to lowercase, remove the stop words and punctuation if set to true at init"""
        words = self.__tokenize_text__(text)
        
        # set words to lowercase and remove punctuation
        words = [word.lower() for word in words if word.isalpha()]
        if self.perform_stemming:
            words = [self.__stem_word__(word) for word in words]
        if self.remove_stopwords:
            words = list(filter(lambda w: w not in self.stopwords, words))
        
        return ' '.join(words)

tp = TextProcessor(remove_stopwords=True, perform_stemming=True)
print(tp.preprocess_text(df[feature_column][0]))

good qualiti dog food


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ludovic\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ludovic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Preparing data for training the models

In [8]:
feature_column_cleanup = feature_column + '_cleanup'
# apply preprocessing operations on the text
def prepare_text(data, feature_column):
    data.loc[:, [feature_column_cleanup]] = data[feature_column].apply(tp.preprocess_text)
    return data
    

X_train = prepare_text(X_train, feature_column)
X_test = prepare_text(X_test, feature_column)

feature_column = feature_column_cleanup

e:\documents\school\swisscom_assignment\finefoodreview\venv\lib\site-packages\pandas\core\indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
e:\documents\school\swisscom_assignment\finefoodreview\venv\lib\site-packages\pandas\core\indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# bag of words + tfidf on text
vectorizer = CountVectorizer()
transformer = TfidfTransformer()

X_train_counts = vectorizer.fit_transform(X_train[feature_column_cleanup])
X_train_tfidf = transformer.fit_transform(X_train_counts)

X_test_counts = vectorizer.transform(X_test[feature_column_cleanup])
X_test_tfidf = transformer.transform(X_test_counts)

print(X_train_tfidf.shape)
print(X_test_tfidf.shape)

(47998, 7761)
(12000, 7761)


### Model

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB

models_dict = {
    'LogisticRegression': LogisticRegression(max_iter=100000),
    'MultinomialNB': MultinomialNB(),
}

In [11]:
from sklearn.metrics import accuracy_score

def compute_metrics(model, X, Y):
    metrics_dict = {}
    y_pred = model.predict(X)
    metrics_dict['accuracy'] = accuracy_score(Y, y_pred)
    
    return metrics_dict

In [12]:
models_results = {}

for label, model in models_dict.items():
    # fit and test the model
    model.fit(X_train_tfidf, Y_train)
    models_results[label] = compute_metrics(model, X_test_tfidf, Y_test)
    
print(models_results)

NameError: name 'metrics_dictr' is not defined

### Plot metrics

In [ ]:
sns.set_theme()

def plot_bar(results_dict, title, ylabel):
    plt.bar(results_dict.keys(), results_dict.values())
    plt.ylabel(ylabel)
    plt.title(title)
    plt.xticks(rotation=30)
    plt.ylim((0,1))
    plt.show()

metrics_dict = {}

for label, results in models_results.items():
    for metric, val in results.items():
        metrics_dict[f"{label}_{metric}"] = val
        
plot_bar(metrics_dict, "Accuracies plot for score prediction", "Accuracy")

### Print the most important words at each score

In [ ]:
TOP_N_WORDS = 5

def get_top_words_score(clf_pipeline, TOP_N):
    feature_names = vectorizer.get_feature_names()
    top_words = {}
    for i, score in enumerate(set(Y_test)):
        # get the TOP_N sorted (desc) by their coefficient of importance in the classifier
        top = np.argsort(models_dict['reg'].coef_[i])[-TOP_N_WORDS:]
        top_words[score] = list(feature_names[word_id] for word_id in top)
        
    return top_words

top_words = get_top_words_score(text_clf, TOP_N = TOP_N_WORDS)
for score, words in top_words.items():
    print(f"[Score = {score}] : {words}")